## 外れ値が多かったり, Loyaltyに特徴があるCity, State, Subsectorを特徴にする

In [8]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

from tqdm import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
from sklearn.metrics import mean_squared_error

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

feat_no = '115_out_'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Data Load

In [16]:
# df = utils.read_pkl_gzip('../stack/1228_003_en_route_500seed_lgb_CV3-6523085700161366_143features.gz')
# for i, col in enumerate(df.columns):
#     if not(col.count('target')):
#         continue
#     df.drop(col, axis=1, inplace=True)
    
# df.head()
train = utils.read_df_pkl('../input/train0*')
test = utils.read_df_pkl('../input/test0*')
# result = df.merge(train, how='left', on=key)

100%|██████████| 3/3 [00:00<00:00, 84.95it/s]


In [ ]:
pred_cols = [col for col in result.columns if col.count('pred')]

result['pred_mean'] = result[pred_cols].mean(axis=1)
result['pred_std'] = result[pred_cols].std(axis=1)
result['pred_max'] = result[pred_cols].max(axis=1)
result['pred_min'] = result[pred_cols].min(axis=1)
result['pred_diff_target'] = result['pred_mean'] - result[target]

In [ ]:
result.drop(pred_cols, axis=1, inplace=True)
result.to_csv('../output/1228_elo_eda_500seed_pred.csv', index=False)

In [12]:
import glob

win_path = '../features/4_winner/*'
# Data Load
base = utils.read_df_pkl('../input/base*')
win_path_list = glob.glob(win_path)
train_path_list = []
test_path_list = []
for path in win_path_list:
    if path.count('train'):
        train_path_list.append(path)
    elif path.count('test'):
        test_path_list.append(path)

base_train = base[~base[target].isnull()].reset_index(drop=True)
base_test = base[base[target].isnull()].reset_index(drop=True)
train_feature_list = utils.parallel_load_data(path_list=train_path_list)
test_feature_list = utils.parallel_load_data(path_list=test_path_list)
train = pd.concat(train_feature_list, axis=1)
train = pd.concat([base_train, train], axis=1)
test = pd.concat(test_feature_list, axis=1)
test = pd.concat([base_test, test], axis=1)

train_test = pd.concat([train, test], ignore_index=True)
train_test.head()

100%|██████████| 3/3 [00:00<00:00, 100.07it/s]


,111_ker_card_id_total@,111_ker_dayofweek@,111_ker_elapsed_time@,111_ker_feature_1@,111_ker_feature_2@,111_ker_feature_3@,111_ker_hist_authorized_flag_mean@,111_ker_hist_authorized_flag_sum@,111_ker_hist_card_id_size@,111_ker_hist_category_1_mean@,111_ker_hist_category_1_sum@,111_ker_hist_category_2_mean_mean@,111_ker_hist_category_3_mean_mean@,111_ker_hist_dayofweek_nunique@,111_ker_hist_first_buy@,111_ker_hist_hour_nunique@,111_ker_hist_installments_max@,111_ker_hist_installments_mean@,111_ker_hist_installments_min@,111_ker_hist_installments_sum@,111_ker_hist_installments_var@,111_ker_hist_merchant_category_id_nunique@,111_ker_hist_merchant_id_nunique@,111_ker_hist_month_diff_mean@,111_ker_hist_month_lag_max@,111_ker_hist_month_lag_mean@,111_ker_hist_month_lag_min@,111_ker_hist_month_lag_var@,111_ker_hist_month_nunique@,111_ker_hist_purchase_amount_max@,111_ker_hist_purchase_amount_mean@,111_ker_hist_purchase_amount_min@,111_ker_hist_purchase_amount_sum@,111_ker_hist_purchase_amount_var@,111_ker_hist_purchase_date_average@,111_ker_hist_purchase_date_diff@,111_ker_hist_purchase_date_max@,111_ker_hist_purchase_date_min@,111_ker_hist_purchase_date_uptonow@,111_ker_hist_subsector_id_nunique@,111_ker_hist_weekend_mean@,111_ker_hist_weekend_sum@,111_ker_hist_weekofyear_nunique@,111_ker_hist_year_nunique@,111_ker_month@,111_ker_new_hist_card_id_size@,111_ker_new_hist_category_1_mean@,111_ker_new_hist_category_1_sum@,111_ker_new_hist_category_2_mean_mean@,111_ker_new_hist_category_3_mean_mean@,111_ker_new_hist_dayofweek_nunique@,111_ker_new_hist_first_buy@,111_ker_new_hist_hour_nunique@,111_ker_new_hist_installments_max@,111_ker_new_hist_installments_mean@,111_ker_new_hist_installments_min@,111_ker_new_hist_installments_sum@,111_ker_new_hist_installments_var@,111_ker_new_hist_merchant_category_id_nunique@,111_ker_new_hist_merchant_id_nunique@,111_ker_new_hist_month_diff_mean@,111_ker_new_hist_month_lag_max@,111_ker_new_hist_month_lag_mean@,111_ker_new_hist_month_lag_min@,111_ker_new_hist_month_lag_var@,111_ker_new_hist_month_nunique@,111_ker_new_hist_purchase_amount_max@,111_ker_new_hist_purchase_amount_mean@,111_ker_new_hist_purchase_amount_min@,111_ker_new_hist_purchase_amount_sum@,111_ker_new_hist_purchase_amount_var@,111_ker_new_hist_purchase_date_average@,111_ker_new_hist_purchase_date_diff@,111_ker_new_hist_purchase_date_max@,111_ker_new_hist_purchase_date_min@,111_ker_new_hist_purchase_date_uptonow@,111_ker_new_hist_subsector_id_nunique@,111_ker_new_hist_weekend_mean@,111_ker_new_hist_weekend_sum@,111_ker_new_hist_weekofyear_nunique@,111_ker_new_hist_year_nunique@,111_ker_purchase_amount_total@,111_ker_weekofyear@,114_mer_merchant_id_M_ID_00a6ca8a8a@,114_mer_merchant_id_M_ID_076cd8a829@,114_mer_merchant_id_M_ID_09d7a200d1@,114_mer_merchant_id_M_ID_1ac6bbc867@,114_mer_merchant_id_M_ID_1f4773aa76@,114_mer_merchant_id_M_ID_2637773dd2@,114_mer_merchant_id_M_ID_26d4fadb60@,114_mer_merchant_id_M_ID_29ec8d2ab8@,114_mer_merchant_id_M_ID_2c4b8c61f6@,114_mer_merchant_id_M_ID_3111c6df35@,114_mer_merchant_id_M_ID_445742726b@,114_mer_merchant_id_M_ID_48257bb851@,114_mer_merchant_id_M_ID_4e461f7e14@,114_mer_merchant_id_M_ID_50f575c681@,114_mer_merchant_id_M_ID_57df19bf28@,114_mer_merchant_id_M_ID_5ba019a379@,114_mer_merchant_id_M_ID_5cea7e02a8@,114_mer_merchant_id_M_ID_5e8220e564@,114_mer_merchant_id_M_ID_620909e2a7@,114_mer_merchant_id_M_ID_69f80607d1@,114_mer_merchant_id_M_ID_6f274b9340@,114_mer_merchant_id_M_ID_77197a9b5e@,114_mer_merchant_id_M_ID_77e2942cd8@,114_mer_merchant_id_M_ID_79692349d6@,114_mer_merchant_id_M_ID_82a30d9203@,114_mer_merchant_id_M_ID_86be58d7e0@,114_mer_merchant_id_M_ID_86ec983688@,114_mer_merchant_id_M_ID_879f625c89@,114_mer_merchant_id_M_ID_8c6b897b35@,114_mer_merchant_id_M_ID_8fadd601d2@,114_mer_merchant_id_M_ID_8fcb16a959@,114_mer_merchant_id_M_ID_9139332ccc@,114_mer_merchant_id_M_ID_940fb4498f@,114_mer_merchant_id_M_ID_96ba3c7b60@,114_mer_merchant_id_M_ID_97f0959810@,114_mer_merchant_id_M_ID_9dbbaabb0a@,114_m

In [21]:
train.sort_index(axis=1, inplace=True)

seed500 = pd.read_csv('../output/1228_elo_eda_500seed_pred.csv')
seed500 = seed500[~seed500[target].isnull()]

seed_max5 = seed500.query("pred_max>=-5 and pred_max<-3 and target >-100")
seed_max5.shape
seed_max5 = seed_max5.merge(train, how='inner', on=key)
seed_max5.shape

(201917, 11)


SystemExit: 

In [35]:
seed_max5.head()

,card_id,first_active_month,feature_1,feature_2,feature_3,target_x,pred_mean,pred_std,pred_max,pred_min,pred_diff_target,111_ker_card_id_total@,111_ker_dayofweek@,111_ker_elapsed_time@,111_ker_feature_1@,111_ker_feature_2@,111_ker_feature_3@,111_ker_hist_authorized_flag_mean@,111_ker_hist_authorized_flag_sum@,111_ker_hist_card_id_size@,111_ker_hist_category_1_mean@,111_ker_hist_category_1_sum@,111_ker_hist_category_2_mean_mean@,111_ker_hist_category_3_mean_mean@,111_ker_hist_dayofweek_nunique@,111_ker_hist_first_buy@,111_ker_hist_hour_nunique@,111_ker_hist_installments_max@,111_ker_hist_installments_mean@,111_ker_hist_installments_min@,111_ker_hist_installments_sum@,111_ker_hist_installments_var@,111_ker_hist_merchant_category_id_nunique@,111_ker_hist_merchant_id_nunique@,111_ker_hist_month_diff_mean@,111_ker_hist_month_lag_max@,111_ker_hist_month_lag_mean@,111_ker_hist_month_lag_min@,111_ker_hist_month_lag_var@,111_ker_hist_month_nunique@,111_ker_hist_purchase_amount_max@,111_ker_hist_purchase_amount_mean@,111_ker_hist_purchase_amount_min@,111_ker_hist_purchase_amount_sum@,111_ker_hist_purchase_amount_var@,111_ker_hist_purchase_date_average@,111_ker_hist_purchase_date_diff@,111_ker_hist_purchase_date_max@,111_ker_hist_purchase_date_min@,111_ker_hist_purchase_date_uptonow@,111_ker_hist_subsector_id_nunique@,111_ker_hist_weekend_mean@,111_ker_hist_weekend_sum@,111_ker_hist_weekofyear_nunique@,111_ker_hist_year_nunique@,111_ker_month@,111_ker_new_hist_card_id_size@,111_ker_new_hist_category_1_mean@,111_ker_new_hist_category_1_sum@,111_ker_new_hist_category_2_mean_mean@,111_ker_new_hist_category_3_mean_mean@,111_ker_new_hist_dayofweek_nunique@,111_ker_new_hist_first_buy@,111_ker_new_hist_hour_nunique@,111_ker_new_hist_installments_max@,111_ker_new_hist_installments_mean@,111_ker_new_hist_installments_min@,111_ker_new_hist_installments_sum@,111_ker_new_hist_installments_var@,111_ker_new_hist_merchant_category_id_nunique@,111_ker_new_hist_merchant_id_nunique@,111_ker_new_hist_month_diff_mean@,111_ker_new_hist_month_lag_max@,111_ker_new_hist_month_lag_mean@,111_ker_new_hist_month_lag_min@,111_ker_new_hist_month_lag_var@,111_ker_new_hist_month_nunique@,111_ker_new_hist_purchase_amount_max@,111_ker_new_hist_purchase_amount_mean@,111_ker_new_hist_purchase_amount_min@,111_ker_new_hist_purchase_amount_sum@,111_ker_new_hist_purchase_amount_var@,111_ker_new_hist_purchase_date_average@,111_ker_new_hist_purchase_date_diff@,111_ker_new_hist_purchase_date_max@,111_ker_new_hist_purchase_date_min@,111_ker_new_hist_purchase_date_uptonow@,111_ker_new_hist_subsector_id_nunique@,111_ker_new_hist_weekend_mean@,111_ker_new_hist_weekend_sum@,111_ker_new_hist_weekofyear_nunique@,111_ker_new_hist_year_nunique@,111_ker_purchase_amount_total@,111_ker_weekofyear@,114_mer_merchant_id_M_ID_00a6ca8a8a@,114_mer_merchant_id_M_ID_076cd8a829@,114_mer_merchant_id_M_ID_09d7a200d1@,114_mer_merchant_id_M_ID_1ac6bbc867@,114_mer_merchant_id_M_ID_1f4773aa76@,114_mer_merchant_id_M_ID_2637773dd2@,114_mer_merchant_id_M_ID_26d4fadb60@,114_mer_merchant_id_M_ID_29ec8d2ab8@,114_mer_merchant_id_M_ID_2c4b8c61f6@,114_mer_merchant_id_M_ID_3111c6df35@,114_mer_merchant_id_M_ID_445742726b@,114_mer_merchant_id_M_ID_48257bb851@,114_mer_merchant_id_M_ID_4e461f7e14@,114_mer_merchant_id_M_ID_50f575c681@,114_mer_merchant_id_M_ID_57df19bf28@,114_mer_merchant_id_M_ID_5ba019a379@,114_mer_merchant_id_M_ID_5cea7e02a8@,114_mer_merchant_id_M_ID_5e8220e564@,114_mer_merchant_id_M_ID_620909e2a7@,114_mer_merchant_id_M_ID_69f80607d1@,114_mer_merchant_id_M_ID_6f274b9340@,114_mer_merchant_id_M_ID_77197a9b5e@,114_mer_merchant_id_M_ID_77e2942cd8@,114_mer_merchant_id_M_ID_79692349d6@,114_mer_merchant_id_M_ID_82a30d9203@,114_mer_merchant_id_M_ID_86be58d7e0@,114_mer_merchant_id_M_ID_86ec983688@,114_mer_merchant_id_M_ID_879f625c89@,114_mer_merchant_id_M_ID_8c6b897b35@,114_mer_merchant_id_M_ID_8fadd601d2@,114_mer_merchant_id_M_ID_8fcb16a959@,114_mer_merchant_id_M_ID_9139332ccc@,114_mer_merchant_id_M_ID_940fb449

In [36]:
seed_max5.to_csv('../output/1228_elo_eda_outlier_max_-5-3.csv', index=False)

In [22]:
id_seed_std001 = seed500[seed500['pred_std']<0.01][key].values
id_seed_max5 = seed500.query("pred_max<-5 and target >-100")[key].values
id_seed_std001 = np.append(id_seed_std001, id_seed_max5)

In [42]:
pd.Series(id_seed_std001).to_csv('../output/1229_elo_id_list_std001_max-5.csv', index=False)

In [32]:
# train.loc[train.card_id.isin(id_seed_std001), :].shape
seed500.query("pred_max<-5 and target >-100")[seed500[target]>-30].shape

(425, 11)